7. Develop a classification model using YOLO object detection algorithm 5 using OpenCV.

In [2]:
import cv2
import numpy as np

In [3]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [6]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [9]:
img = cv2.imread("car.jpg")
height, width, channels = img.shape
blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),swapRB=True, crop=False)

In [10]:
net.setInput(blob)
outs = net.forward(output_layers)

In [11]:
class_ids = []
confidences = []
boxes = []

In [15]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores) 
        confidence = scores[class_id] 
        if confidence > 0.5:
            center_x = int(detection[0] * width) 
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2) 
            boxes.append([x, y, w, h]) 
            confidences.append(float(confidence)) 
            class_ids.append(class_id)

In [16]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [17]:
for i in range(len(boxes)): 
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) 
        cv2.putText(img, label, (x, y -5),cv2.FONT_HERSHEY_SIMPLEX,1/2, color, 2)
cv2.imshow("Image",img) 
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1